## Import data and format

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
!pip install datasets

import pandas as pd
from datasets import Dataset

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [4]:
data = pd.read_csv('https://raw.githubusercontent.com/laxmimerit/All-CSV-ML-Data-Files-Download/master/IMDB-Dataset.csv')
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [5]:
dataset = Dataset.from_pandas(data)
dataset = dataset.train_test_split(test_size=0.3)
dataset

DatasetDict({
    train: Dataset({
        features: ['review', 'sentiment'],
        num_rows: 35000
    })
    test: Dataset({
        features: ['review', 'sentiment'],
        num_rows: 15000
    })
})

In [6]:
dataset.column_names

{'train': ['review', 'sentiment'], 'test': ['review', 'sentiment']}

In [7]:
data['sentiment'].value_counts()

,count
sentiment,
positive,25000
negative,25000


In [8]:
label2id = {'negative': 0,
            'positive': 1
            }
id2label = {0: 'negative',
            1: 'positive'}
dataset = dataset.map(lambda x: {'label': label2id[x['sentiment']]})


Map:   0%|          | 0/35000 [00:00<?, ? examples/s]

Map:   0%|          | 0/15000 [00:00<?, ? examples/s]

In [9]:
dataset['train'][0]

{'review': 'I have given this film an elevated rating of 2 stars as I personally appear in minutes 42 and 43 of the film....the road side bar scene in Russia. In this scene the director of the movie offered me the immortal line - "50 Dollars..you Drink and Talk", but I felt that my Polish counterpart could speak in a more convincing Russian accent than I could, so I declined to take this speaking part on. I was slightly starstruck as this was my first Film experience....and who knows... these lines could have ended up there with lines such as "I\'ll be Back" and "Quite Frankly My Dear, I Don\'t Give a Damn". Had I spoken that one line then my name would appear in the credits of Rancid Aluminium as \'Heavy 1\' instead of the name of Ryszard Janikowski. <br /><br />As time goes on, I am counting myself lucky that my name is in no way connected to this film.<br /><br />Even though I spent a whole day on the set, in South Wales hot-spot Barry Island, no one could tell me what the actual st

## Tokenize the inputs

In [10]:
from transformers import AutoTokenizer
import torch

In [11]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model_ckpt = 'huawei-noah/TinyBERT_General_4L_312D'
tokenizer = AutoTokenizer.from_pretrained(model_ckpt, use_fast=True)

config.json:   0%|          | 0.00/409 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [12]:
print(dataset['train'][0])
print(tokenizer(dataset['train'][0]['review']))

{'review': 'I have given this film an elevated rating of 2 stars as I personally appear in minutes 42 and 43 of the film....the road side bar scene in Russia. In this scene the director of the movie offered me the immortal line - "50 Dollars..you Drink and Talk", but I felt that my Polish counterpart could speak in a more convincing Russian accent than I could, so I declined to take this speaking part on. I was slightly starstruck as this was my first Film experience....and who knows... these lines could have ended up there with lines such as "I\'ll be Back" and "Quite Frankly My Dear, I Don\'t Give a Damn". Had I spoken that one line then my name would appear in the credits of Rancid Aluminium as \'Heavy 1\' instead of the name of Ryszard Janikowski. <br /><br />As time goes on, I am counting myself lucky that my name is in no way connected to this film.<br /><br />Even though I spent a whole day on the set, in South Wales hot-spot Barry Island, no one could tell me what the actual st

In [13]:
def tokenize(batch):
    temp = tokenizer(batch['review'], padding=True, truncation=True, max_length=300)
    return temp

dataset = dataset.map(tokenize, batched=True, batch_size=None)

Map:   0%|          | 0/35000 [00:00<?, ? examples/s]

Map:   0%|          | 0/15000 [00:00<?, ? examples/s]

In [14]:
dataset['train'][0].keys()

dict_keys(['review', 'sentiment', 'label', 'input_ids', 'token_type_ids', 'attention_mask'])

## Model Evaluation

In [15]:
!pip install evaluate

import evaluate
import numpy as np
import sklearn

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.5 MB/s eta 0:00:00


In [16]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(model_ckpt,
                                                           num_labels=len(label2id),
                                                           label2id=label2id,
                                                           id2label=id2label)

pytorch_model.bin:   0%|          | 0.00/62.7M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at huawei-noah/TinyBERT_General_4L_312D and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
args = TrainingArguments(
    output_dir='train_dir',
    overwrite_output_dir=True,
    num_train_epochs=3,
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    evaluation_strategy='epoch'
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
)

In [18]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [19]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.352100,0.330054,0.861800
2,0.290800,0.299191,0.872267
3,0.256200,0.291453,0.880067


TrainOutput(global_step=3282, training_loss=0.3169583094548627, metrics={'train_runtime': 668.7852, 'train_samples_per_second': 157.001, 'train_steps_per_second': 4.907, 'total_flos': 882184338000000.0, 'train_loss': 0.3169583094548627, 'epoch': 3.0})

In [20]:
trainer.evaluate()

{'eval_loss': 0.2914528548717499,
 'eval_accuracy': 0.8800666666666667,
 'eval_runtime': 29.7977,
 'eval_samples_per_second': 503.394,
 'eval_steps_per_second': 15.739,
 'epoch': 3.0}

In [21]:
trainer.save_model('tinybert-sentiment-analysis')

In [22]:
data = ['this movie was horrible, the plot was really boring. acting was okay',
        'the movie is really sucked. there is not plot and acting was bad',
        'what a beautiful movie. great plot. acting was good. will see it again']

In [23]:
from transformers import pipeline
import torch

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

classifier = pipeline('text-classification', model='tinybert-sentiment-analysis', device=device)

classifier(data)

[{'label': 'negative', 'score': 0.9912823438644409},
 {'label': 'negative', 'score': 0.9912341237068176},
 {'label': 'positive', 'score': 0.9893152713775635}]